In [1]:
!pip3 install tflearn

import numpy as np
from scipy.misc import imread, imresize
from glob import glob # helps find files in a folder
import os  # to navigate through filesystem
from google.colab import drive # needed to connect to drive
import matplotlib.pyplot as plt # need to plot images
import os, sys
import glob
from scipy.misc import imread, imresize, imshow
from tflearn.layers.core import input_data
from tflearn.layers.conv import global_avg_pool, conv_1d, max_pool_1d
from tflearn.layers.core import fully_connected
from tflearn.layers.estimator import regression

    100% |████████████████████████████████| 102kB 3.8MB/s 
  Running setup.py bdist_wheel for tflearn ... - \ done
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir('drive/My Drive/concatenated')

In [0]:
file = open('test neg.txt', 'r')
text = file.read()
text = text.split('--sep--')
test_neg = text[1:]
assert (len(test_neg) == 12500), 'test neg error'

file = open('test pos.txt', 'r')
text = file.read()
text = text.split('--sep--')
test_pos = text[1:]
assert (len(test_pos) == 12500), 'test pos error'

file = open('train pos.txt', 'r')
text = file.read()
text = text.split('--sep--')
train_pos = text[1:]
assert (len(train_pos) == 12500), 'train pos error'

file = open('train neg.txt', 'r')
text = file.read()
text = text.split('--sep--')
train_neg = text[1:]
assert (len(train_neg) == 12500), 'train neg error'

file.close()

for i in range(12500):
    test_neg[i]=test_neg[i].replace('\n', '')
    test_pos[i]=test_pos[i].replace('\n', '')
    train_neg[i]=train_neg[i].replace('\n', '')
    train_pos[i]=train_pos[i].replace('\n', '')
    test_neg[i]=test_neg[i].replace('<br />', '\n')
    test_pos[i]=test_pos[i].replace('<br />', '\n')
    train_neg[i]=train_neg[i].replace('<br />', '\n')
    train_pos[i]=train_pos[i].replace('<br />', '\n')

In [0]:
t_pos=np.asarray(train_pos).view(np.uint8)

In [0]:
max_len=len(max(train_pos+train_neg+test_pos+test_neg, key=len))

In [8]:
max_len

13604

In [0]:
train=np.zeros([25000, max_len])

In [0]:
for idx,item in enumerate(train_pos+train_neg):
    for idx2,letter in enumerate(item):
        train[idx,idx2]=ord(letter)

In [11]:
train.shape

(25000, 13604)

In [12]:
train = train[...,:1000]
train.shape

(25000, 1000)

In [13]:
train

array([[ 66., 114., 111., ...,   0.,   0.,   0.],
       [ 73., 102.,  32., ...,   0.,   0.,   0.],
       [ 73.,  39., 109., ...,   0.,   0.,   0.],
       ...,
       [ 67.,  97., 112., ..., 108.,  97., 110.],
       [ 84., 111., 119., ..., 105., 110., 101.],
       [ 84., 104., 105., ...,   0.,   0.,   0.]])

In [14]:
input_layer=input_data([None, 1000, 1])

conv1=conv_1d(input_layer, nb_filter = 64, filter_size = (15, 1), strides=[2,1], activation='relu')

pool1=max_pool_1d(conv1, kernel_size=[3,1], strides=[2,1])


conv2=conv_1d(pool1, nb_filter = 128, filter_size = (14, 1), strides=[2,1], activation='relu')

pool2=max_pool_1d(conv2, kernel_size=[2,1], strides=[2,1])


conv3=conv_1d(pool2, nb_filter = 256, filter_size = (11, 1), strides=[2,1], activation='relu')


conv4=conv_1d(conv3, nb_filter = 512, filter_size = (9, 1), strides=[1,1], activation='relu')

out_layer = fully_connected(conv4, 2, activation='softmax')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [15]:
conv1

<tf.Tensor 'Conv1D/Relu:0' shape=(?, 500, 64) dtype=float32>

In [16]:
pool1

<tf.Tensor 'MaxPool1D/Squeeze:0' shape=(?, 250, 64) dtype=float32>

In [17]:
conv2

<tf.Tensor 'Conv1D_1/Relu:0' shape=(?, 125, 128) dtype=float32>

In [18]:
pool2

<tf.Tensor 'MaxPool1D_1/Squeeze:0' shape=(?, 63, 128) dtype=float32>

In [19]:
conv3

<tf.Tensor 'Conv1D_2/Relu:0' shape=(?, 32, 256) dtype=float32>

In [20]:
conv4

<tf.Tensor 'Conv1D_3/Relu:0' shape=(?, 32, 512) dtype=float32>

In [21]:
out_layer

<tf.Tensor 'FullyConnected/Softmax:0' shape=(?, 2) dtype=float32>

In [22]:
network = regression(out_layer, optimizer='adam',
                     loss='categorical_crossentropy', learning_rate=0.001)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [0]:
import tflearn

In [0]:
model = tflearn.DNN(network, tensorboard_verbose=2, tensorboard_dir='./Graph')

In [0]:
import tensorflow as tf  # tensorflow during training
from tflearn.activations import relu # rectified linear activation function

# normalizes data 
from tflearn.layers.normalization import batch_normalization as bn
from scipy.misc import imread, bytescale, imresize  # image manipulation functions
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d, global_avg_pool
from tflearn.layers.estimator import regression  # trainer for the network

# turns scalar label into vector where appropriate class is value 1 and others 0
from tflearn.data_utils import to_categorical  

# makes new images from the ones we have by flipping them, rotating, etc.
from tflearn.data_augmentation import ImageAugmentation

# to visualize the data space
from tensorflow.contrib.tensorboard.plugins import projector

In [0]:
Y = np.zeros([25000,2])
Y[:12500,1]=1
Y[12500:,0]=1

In [0]:
train = train[...,None]

In [28]:
!pip install tensorboardcolab

  Running setup.py bdist_wheel for tensorboardcolab ... - done
  Stored in directory: /root/.cache/pip/wheels/ab/74/02/cda602d1dc28b2f12eab313c49b9bfa14d6371326bc2590e06
Successfully built tensorboardcolab


In [29]:
from tensorboardcolab import *

Using TensorFlow backend.


In [30]:
tcb = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://273c6d43.ngrok.io


In [31]:
tX=np.zeros([25000, max_len])
for idx,item in enumerate(test_pos+test_neg):
    for idx2,letter in enumerate(item):
        tX[idx,idx2]=ord(letter)

tX = tX[...,:1000]
tX.shape

(25000, 1000)

In [0]:
tY = np.zeros([25000,2])
tY[:12500,1]=1
tY[12500:,0]=1

In [0]:
tX = tX[...,None]

In [34]:
model.fit(train, Y, n_epoch=18, shuffle=True, validation_set = (tX,tY), show_metric=True, batch_size=250, run_id='first_run3')

Training Step: 1799  | total loss: 11.59782 | time: 12.400s
| Adam | epoch: 018 | loss: 11.59782 - acc: 0.4963 -- iter: 24750/25000
Training Step: 1800  | total loss: 11.48802 | time: 14.722s
| Adam | epoch: 018 | loss: 11.48802 - acc: 0.5011 | val_loss: 11.51293 - val_acc: 0.5000 -- iter: 25000/25000
--


In [0]:
    model.save('models/1000_chars_2')

INFO:tensorflow:/content/drive/My Drive/concatenated/models/1000_chars is not in all_model_checkpoint_paths. Manually adding it.
